In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

import sys

import grade_tools as gt

import seaborn as sns

import os


# For mailing
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

import base64
import ssl
import getpass

import datetime as dt


# Canvas grades

In [ ]:
#infile = 'phys220f23.csv'
#course = "PHYS 220 Modern Physics"

infile = 'phys260s24.csv'
course = "PHYS 260 Thermal Physics"


tag = course.replace(' ','')
print(f"course: {course}")
print(f"tag: {tag}")

df = pd.read_csv(infile)

# Replace nans with 0
df = df.fillna(0)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
# Organize some columns

nentries = len(df)
lname = []
fname = []
for i in range(nentries):
    name = df.iloc[i]['Student']
    if name.find(',')>=0:
        l,f = name.split(',')
        lname.append(l.replace(' ',''))
        fname.append(f.replace(' ',''))
    else:
        lname.append('NaN')
        fname.append('NaN')
df['First Name'] = fname
df['Last Name'] = lname

#### EMAIL

df = df.rename(columns={'SIS Login ID':'Email Address'})
df

In [ ]:
df['Email Address']

In [ ]:
df.columns

In [ ]:
import textwrap
def wrap_labels(ax, width, break_long_words=False):
    labels = []
    for label in ax.get_xticklabels():
        text = label.get_text()
        labels.append(textwrap.fill(text, width=width,
                      break_long_words=break_long_words))
    ax.set_xticklabels(labels, rotation=270)

In [ ]:
def represents_float(s):
    try: 
        float(s)
    except ValueError:
        return False
    else:
        return True

In [ ]:
def get_assignments(df):
    
    day_org = dt.datetime(2023,9,1)
    #day = day_org
    
    # First entry is what we want
    # Second entry is what we *don't* want
    maps = {}


    # This was for PHYS 220, Modern Physics, F23
    '''
    maps['quiz'] = [['QUIZ'],['PRE-READING']]
    maps['hw'] = [['HOMEWORK'],[]]
    maps["participation"] = [['PARTICIPATION', 'participation'], []]
    #maps['final'] = [['FINAL EXAM', 'Final exam - Day 2'],[]]
    maps['final1'] = [['Final exam - Day 1'],[]]
    maps['final2'] = [['Final exam - Day 2'],[]]
    maps['readingquiz'] = [['PRE-READING', 'READING'],[]]
    maps['lab'] = [['Lab grade'],['Current Score', 'Final Score']]
    maps['reflection'] = [['REFLECTION PAPER'],['Current Score']]
    maps['optional'] = [['OPTIONAL EXTRA WORK'],[]]
    '''

    # For PHYS 260, Thermal Physics, S24
    maps['quiz'] = [['QUIZ'],['REMINDER']]
    maps['hw'] = [['HOMEWORK'],[]]
    #maps["participation"] = [['PARTICIPATION', 'participation'], []]
    #maps['final'] = [['FINAL EXAM', 'Final exam - Day 2'],[]]
    #maps['final1'] = [['Final exam - Day 1'],[]]
    #maps['final2'] = [['Final exam - Day 2'],[]]
    #maps['readingquiz'] = [['PRE-READING', 'READING'],[]]
    #maps['lab'] = [['Lab grade'],['Current Score', 'Final Score']]
    #maps['reflection'] = [['REFLECTION PAPER'],['Current Score']]
    #maps['optional'] = [['OPTIONAL EXTRA WORK'],[]]

    colnames = df.columns

    assignments = {"name":[], "points":[], "date":[], "type":[]}
    
    for name in colnames:
        #print(name, " -----------------")
        for key in maps.keys():
            #print(key)
            MATCH = False
            
            matches = []
            match_key = None
            for val_true in maps[key][0]:
                # Does it match with the first one
                if name.find(val_true)>=0:
                    MATCH = True
                    # Does it *not* match with the second
                    for val_false in maps[key][1]:
                        if name.find(val_false)>=0:
                            MATCH = False
                if MATCH:
                    match_key = key
                    break
            #print(MATCH,name,key,val,"\n")            
            if MATCH:
                # This is where we find the total points for this assignment
                points = df[name].iloc[0]
                #print(f"{type(points)} ------------------------")
                assignments['name'].append(name)
                assignments['type'].append(match_key)
                print(name)
                assignment_ID = name.split('(')[1].split(')')[0]
                print(assignment_ID)
                day = day_org + dt.timedelta(seconds=20*int(assignment_ID))

                #assignments['date'].append(df[name].iloc[0])
                assignments['date'].append(pd.to_datetime(day))
                
                points = float(points)
                if points != points:
                    points = 0
                assignments['points'].append(float(points))

    
    dfa = pd.DataFrame.from_dict(assignments)
    return dfa
            

dfa = get_assignments(df)

dfa

dfa = dfa.sort_values(by='date', ascending=True)
dfa

In [ ]:
# Drop a few assignments that haven't been graded yet
dfa.drop(dfa.loc[dfa['name'] == 'HOMEWORK - Chapter 3b (315965)'].index, inplace=True)
dfa.drop(dfa.loc[dfa['name'] == 'HOMEWORK - Chapter 4 (316499)'].index, inplace=True)

In [ ]:
#dfa['date']

In [ ]:
#dfa['points']
mask = dfa['type'] == 'quiz'
dfa[mask]

In [ ]:
mask = dfa['type'] == 'hw'
dfa[mask]

In [ ]:
names = dfa['name']

dfplot = {"name":[], "scores":[], "type":[]}

for name in names:
    #print(name)
    dftemp = df[name]
    x = dftemp.values[1:] # Skip first row which is total points
    #print(x[0],represents_float(x[0]))
    if represents_float(x[0]):
        x = x.astype(float)
        #print(x)
        #print(len(x))
        if x[0]==x[0]:

            points = dfa[dfa['name']==name]['points'].values[0]
    
            
            dfplot["name"] += len(x)*[name]
            dfplot['scores'] += (x/points).tolist()
            
            atype = dfa[dfa['name']==name]['type'].values[0]
            dfplot['type'] += len(x)*[atype]

            #plt.figure()
            #plt.hist(x)

dfplot = pd.DataFrame.from_dict(dfplot)
dfplot

In [ ]:
plt.figure(figsize=(16,4))

sns.violinplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

plt.xticks(rotation=90);

In [ ]:
plt.figure(figsize=(16,4))

sns.boxplot(data=dfplot, y="scores", x="name", hue='type');

plt.xticks(rotation=90);

In [ ]:
plt.figure(figsize=(16,4))

sns.boxplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

plt.plot([0],[0.8],'ro',markersize=10);

plt.xticks(rotation=270,ha='left');

In [ ]:
anames = np.unique(dfplot['name'])

plt.figure(figsize=(16,4))

sns.boxplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

xlabels = plt.gca().get_xticklabels(which='both')

n = 2

xpts = []
ypts = []

output = ""
for idx,xlabel in enumerate(xlabels):
    print(xlabel)
    name = xlabel.get_text()
    print(name)
    org_score = float(df.iloc[n][name])
    points = dfa[dfa['name']==name]['points'].values[0]
    score = org_score/points
    xpts.append(name)
    ypts.append(score)
    
    
lname = df.iloc[n]['Last Name']
fname = df.iloc[n]['First Name']
email = df.iloc[n]['Email Address']

print()

plt.plot(xpts,ypts,'ro-',markersize=10, label=f'{fname} {lname}');

plt.xticks(rotation=270,ha='left');
plt.legend()





In [ ]:
anames = np.unique(dfplot['name'])

plt.figure(figsize=(16,4))

sns.boxplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

xlabels = plt.gca().get_xticklabels(which='both')

n = 1

xpts = []
ypts = []

output = ""
for idx,xlabel in enumerate(xlabels):
    print(xlabel)
    name = xlabel.get_text()
    print(name)
    org_score = float(df.iloc[n][name])
    points = dfa[dfa['name']==name]['points'].values[0]
    score = org_score/points
    xpts.append(name)
    ypts.append(score)
    
    
lname = df.iloc[n]['Last Name']
fname = df.iloc[n]['First Name']
email = df.iloc[n]['Email Address']

print()

plt.plot(xpts,ypts,'ro-',markersize=10, label=f'{fname} {lname}');

plt.xticks(rotation=270,ha='left');
plt.legend()





In [ ]:
df.iloc[1]

In [ ]:
# Final Day 1 20%  Final Day 2 80% of final 

# PHYS 220, Fall 23
#weighting = {"quiz":0.35, "hw": 0.20, "readingquiz":0.05, "final1": 0.04,"final2": 0.16, "reflection":0.075, "lab":0.125, "optional": 0.0}

# PHYS 260, Spring 24
weighting = {"quiz":0.45, "hw": 0.20, "final1": 0.30, "contributions": 0.05, "optional": 0.0}

weighting_sum = 0
for key in weighting.keys():
    weighting_sum += weighting[key]
    
print(f"Weighting sum: {weighting_sum}")


In [ ]:
n = 12

dfstudent = df.iloc[n]

#print(dfstudent)
#print()

nassignments = len(dfa)

scores = {}
for key in weighting.keys():
    scores[key] = []
print(scores)

for n in range(nassignments):
    aname = dfa.iloc[n]['name']
    atype = dfa.iloc[n]['type']
    points = dfa.iloc[n]['points']
    date = dfa.iloc[n]['date']
    
    org_score = float(dfstudent[aname])
    if org_score != org_score:
        continue
        
    score = org_score / points
    scores[atype].append(score)
    
    
print(scores)
dfstudent

In [ ]:
#dfstudent.keys()

In [ ]:
def average_scores(scores, weighting, drop_lowest=None):
    weight_sum = 0
    what_was_included = []
    
    averages = {}
    
    if drop_lowest is None:
        drop_lowest = {}
        for key in weighting.keys():
            drop_lowest[key] = 0
    
    tot = 0
    
    #if drop_lowest:
    #    scores = n
    
    for key in scores:
        if len(scores[key])>0:
            if key not in what_was_included:
                weight_sum += weighting[key]
                what_was_included.append(key)
            scores_to_use = scores[key]
            #print(type(scores_to_use))
            #print("scores to use: ",scores_to_use)
            #print(drop_lowest)
            #print(key)
            #print(scores_to_use)
            
            #for score in scores_to_use:
            #    print(score, type(score))
            
            # Remove nan's from the list of values
            #while np.nan in scores_to_use:
            #    scores_to_use.remove(np.nan)

            #while pd.np.nan in scores_to_use:
            #    scores_to_use.remove(pd.np.nan)

            # Remove nan's, NaN's
            scores_to_use = np.array(scores_to_use)
            scores_to_use = scores_to_use[scores_to_use==scores_to_use]
                
            if key in drop_lowest.keys():
                if drop_lowest[key]>0:
                    scores_to_use = np.sort(scores_to_use)[drop_lowest[key]:]
            
            ave = np.average(scores_to_use)
            w = weighting[key]
            
            averages[key] = ave
            
            #print(key,scores_to_use, ave, w)

            tot += ave*w
    
    #print(tot, weight_sum)
    tot /= weight_sum
    #print(tot)
    return tot, averages


# Test 1 ###############################
scores = {"hw": [50, 100], "quiz": [45, 85, 96]}
weighting = {"hw": 0.4, "quiz": 0.6}
drops = {"hw": 0, "quiz": 0}

tot,ave = average_scores(scores=scores, weighting=weighting, drop_lowest=drops)
#print(tot,ave)
print(f"Total:  {tot:.2f}")
for k,a in ave.items():
    print(f"{k:6s}: {a:.2f}")
print()
    
# Test 2 ###############################
scores = {"hw": [50, 100], "quiz": [45, 85, 96]}
weighting = {"hw": 0.4, "quiz": 0.6}
drops = {"hw": 0, "quiz": 1}

tot,ave = average_scores(scores=scores, weighting=weighting, drop_lowest=drops)
#print(tot,ave)
print(f"Total:  {tot:.2f}")
for k,a in ave.items():
    print(f"{k:6s}: {a:.2f}")
print()

# Test 3 #################################
scores = {"hw": [50, 100, np.nan], "quiz": [45, 85, 96]}
weighting = {"hw": 0.4, "quiz": 0.6}
drops = {"hw": 0, "quiz": 0}

tot,ave = average_scores(scores=scores, weighting=weighting, drop_lowest=drops)
#print(tot,ave)
print(f"Total:  {tot:.2f}")
for k,a in ave.items():
    print(f"{k:6s}: {a:.2f}")
print()

In [ ]:
################################################################################
def write_and_send_email(sender_email, rec_email, title, name, body_text, subject, attachment=None):

    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = sender_email
    message["To"] = rec_email

    message["Cc"] = "mbellis@siena.edu"

    text = ""
    text += body_text

    part1 = MIMEText(text, "plain")

    message.attach(part1)
    
    # Attachment
    # https://stackoverflow.com/questions/69901601/send-email-with-attachment-using-python-3-7-3
    if attachment is not None:
        filename = attachment
        with open(filename, "rb") as fs:
            part = MIMEBase('application', "octet-stream")
            part.set_payload(fs.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(filename))
        message.attach(part)


    context = ssl.create_default_context()
    
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)

        # UNCOMMENT THIS WHEN SENDING FOR REAL
        receiver_email = rec_email

        server.sendmail(sender_email, receiver_email, message.as_string())





In [ ]:
weighting = {"quiz":0.35, "hw": 0.20, "readingquiz":0.05, "final1": 0.04,"final2": 0.16, "reflection":0.075, "lab":0.125, "optional": 0.0}

#weighting = {"quiz":0.35, "hw": 0.20, "readingquiz":0.05, "final": 0.20, "paper":0.075, "lab":0.125}

drops = {"quiz":1, "hw": 1, "readingquiz":5}


print(average_scores(scores,weighting))

print("---------------")

print(average_scores(scores, weighting, drop_lowest=drops))

In [ ]:
dfstudent

In [ ]:
dfplot.iloc[10]

In [ ]:
df.iloc[10][8]

In [ ]:
dfplot

In [ ]:
df.iloc[23:]

In [ ]:
# Make a directory if we need to for the figures
try:
   os.makedirs("figures")
except FileExistsError:
   # directory already exists
   pass


####################################
# For mailing

########################################################################
login = "1a2b3c4d5e6f7g" # paste your login generated by Mailtrap
password = "1a2b3c4d5e6f7g" # paste your password generated by Mailtrap
sender_email = "matthew.bellis@gmail.com"
port = 465
password = getpass.getpass("Your password: ")

###############################################

d_students = {}
d_students['last name'] = []
d_students['first name'] = []
d_students['course grade (no drops)'] = []
d_students['course grade (with drops)'] = []
for w,v in weighting.items():
    key = f"{w} (no drops)"
    d_students[key] = []
    key = f"{w} (with drops)"
    d_students[key] = []


max_names = 24 
#max_names = 5

# PHYS 220, Fall 23
#weighting = {"quiz":0.35, "hw": 0.20, "readingquiz":0.05, "final1": 0.04,"final2": 0.16, "reflection":0.075, "lab":0.125, "optional": 0.0}
#drops = {"quiz":1, "hw": 1, "readingquiz":5}

# PHYS 260, Spring 24
weighting = {"quiz":0.45, "hw": 0.20, "final1": 0.30, "contributions": 0.05, "optional": 0.0}
drops = {"quiz":0, "hw": 0}


#drops = {"quiz":1, "hw": 1, "participation":0, "final": 0}

course_grades = []

for n in range(1,max_names):
#for n in range(23,24):

    plt.figure(figsize=(32,8), dpi=200, facecolor='white')

    sns.boxplot(data=dfplot, y="scores", x="name", hue='type', dodge=False, width=0.5);

    xlabels = plt.gca().get_xticklabels(which='both')

    xpts = []
    ypts = []
    
    dfstudent = df.iloc[n]

    scores = {}
    for key in weighting.keys():
        scores[key] = []
    #print(scores)
    
    running_aves = []
    all_averages = []
    
    lname = dfstudent['Last Name']
    fname = dfstudent['First Name']
    email = dfstudent['Email Address']

    email_body = f"Summary grades for {fname} {lname} ({email})\n\n"
    
    d_students['last name'].append(lname)
    d_students['first name'].append(fname)
    
    # https://stackoverflow.com/questions/7011992/python-how-to-change-email-text-typeface
    #text = '<font face="Courier New, Courier, monospace">' + text + '</font>'
    #email_body += '<font face="Courier New, Courier, monospace">'
    #email_body += '<html><font="Fixed Width">\n'
    
    for idx,xlabel in enumerate(xlabels):
        #print(xlabel)
        name = xlabel.get_text()
        #print(name)
        org_score = float(dfstudent[name])
        
        '''
        Modern, F23
        ##########################################################
        # Extra credit for final exam question
        ##########################################################
        print("name: ",name)
        if name.find('Final exam - Day 2')>=0:
            print("----------------------------")
            #extra_credit_points = dfa[dfa['name']=='OPTIONAL EXTRA WORK to contribute to final exam (306402)']['points'].values[0]
            extra_credit_points = float(dfstudent['OPTIONAL EXTRA WORK to contribute to final exam (306402)'])
            if extra_credit_points==extra_credit_points and extra_credit_points>=0:
                print(extra_credit_points)
                print(org_score)
                org_score += extra_credit_points
                print(org_score)
        '''
        
        points = dfa[dfa['name']==name]['points'].values[0]
        score = org_score/points
        xpts.append(name)
        ypts.append(score)
        
        #print(xlabel)
        atype = dfa[dfa['name']==xlabel.get_text()]['type'].values[0]
        
        # Google Classroom
        #date = dfa[dfa['name']==xlabel.get_text()]['date'].values[0]
        
        # Canvas
        date = pd.to_datetime(dfa[dfa['name']==xlabel.get_text()]['date'].values[0])

        #print(atype,date)
        scores[atype].append(score)
        running_ave, all_averages_temp = average_scores(scores,weighting)#, drop_lowest=drops)
        running_aves.append(running_ave)
        all_averages.append(all_averages_temp)


        # Google Classroom
        #email_body += f"{100*score:6.2f} ({org_score:5.1f}/{points:5.1f})  {atype:10s} {date:15s} {name:30s} \n"   
        # Canvas
        #print(date)
        email_body += f"{100*score:6.2f} ({org_score:5.1f}/{points:5.1f})  {atype:11s} {date:%m-%d-%Y} {name:30s} \n\n"   
        #break
        

    course_grade, final_all_averages = average_scores(scores, weighting, drop_lowest=drops)

    email_body += "\nDROPPED GRADES HAVE NOT BEEN APPLIED"
    email_body += "\n"
    #email_body += f"Reading quiz  (avg):    {100*final_all_averages['readingquiz']:6.2f}%\n"
    email_body += f"Homeworks     (avg):    {100*final_all_averages['hw']:6.2f}%\n"
    email_body += f"Quizzes       (avg):    {100*final_all_averages['quiz']:6.2f}%\n"
    #email_body += f"Reflection paper   :    {100*final_all_averages['reflection']:6.2f}%\n"
    #email_body += f"Lab grade          :    {100*final_all_averages['lab']:6.2f}%\n"
    #email_body += f"Final exam Day 1   :    {100*final_all_averages['final1']:6.2f}%\n"
    #email_body += f"Final exam Day 2   :    {100*final_all_averages['final2']:6.2f}%\n"

    # Add to dictionary
    d_students['course grade (with drops)'].append(100*course_grade)
    for w,v in weighting.items():
        key = f"{w} (with drops)"
        if w in final_all_averages.keys():
            d_students[key].append(100*final_all_averages[w])
        else:
            d_students[key].append(-1)

    #course_grades.append(course_grade)

    
    email_body += f"\nCourse grade: {100*course_grade:8.2f}%\n"
    #email_body += '</font>'
    #email_body += "</html>"


    plt.plot(xpts,ypts,'k*',markersize=30, linewidth=5);    
    plt.plot(xpts,ypts,'r*',markersize=20, linewidth=5, label=f'{lname}: Assignment grades');
    plt.plot(xpts,running_aves,'ko--',markersize=10, linewidth=5, label=f'{lname}: Course grade {100*course_grade:0.1f}');

    #plt.xticks(rotation=270,ha='center',fontsize=14);
    #plt.xticks(rotation=90,ha='center',fontsize=14);
    #plt.xticks(rotation=90);
    
    plt.legend(loc='lower left')
    
    wrap_labels(plt.gca(), 32)
    
    plt.xlabel('')
    plt.ylabel('Assignment grade',fontsize=14)
    plt.gca().set_facecolor("white")

    plt.tight_layout()
    #plt.gcf().subplots_adjust(bottom=0.5)
    
    ##########################################################################
    # Recalculate with no drops
    
    course_grade, final_all_averages = average_scores(scores, weighting)
    d_students['course grade (no drops)'].append(100*course_grade)
    for w,v in weighting.items():
        key = f"{w} (no drops)"
        if w in final_all_averages.keys():
            d_students[key].append(100*final_all_averages[w])
        else:
            d_students[key].append(-1)
    
    email_body += "\n\nBefore grades were dropped..."
    email_body += "\n"
    #email_body += f"Reading quiz  (avg):    {100*final_all_averages['readingquiz']:6.2f}%\n"
    email_body += f"Homeworks     (avg):    {100*final_all_averages['hw']:6.2f}%\n"
    email_body += f"Quizzes       (avg):    {100*final_all_averages['quiz']:6.2f}%\n"
    #email_body += f"Reflection paper   :    {100*final_all_averages['reflection']:6.2f}%\n"
    #email_body += f"Lab grade          :    {100*final_all_averages['lab']:6.2f}%\n"
    #email_body += f"Final exam Day 1   :    {100*final_all_averages['final1']:6.2f}%\n"
    #email_body += f"Final exam Day 2   :    {100*final_all_averages['final2']:6.2f}%\n"
    email_body += f"\nCourse grade: {100*course_grade:8.2f}%\n"
    ##########################################################################
    
    print(email_body)
    
    outfilename = f"figures/{tag}_summary_{fname}_{lname}.png"
    plt.savefig(outfilename)
    
    subject = f"{course}: summary grades for {fname} {lname}"


    rec_email = email
    # For testing
    #rec_email = 'mbellis@siena.edu'
    #write_and_send_email(sender_email=sender_email, rec_email=rec_email, title=None, name=None, \
    #                     subject=subject, body_text=email_body, attachment=outfilename)
    
df_students = pd.DataFrame.from_dict(d_students)

df_students

In [ ]:
mask = df_students['first name'] == 'Payton'
df_students[mask]

In [ ]:
df_students

In [ ]:
df_students.hist('course grade (with drops)',range=(60,100))

In [ ]:
df_students.plot.scatter(x='quiz (no drops)',y='final2 (no drops)')
#df_students.plot.scatter(x='quiz (with drops)',y='final2 (no drops)')

plt.xlim(30,110)
plt.ylim(30,110)

In [ ]:
df_students.plot.scatter(x='quiz (no drops)',y='quiz (with drops)')

plt.plot([30,110],[30,110],'k--')

plt.xlim(30,110)
plt.ylim(30,110)

In [ ]:
df_students.plot.scatter(x='quiz (no drops)',y='course grade (with drops)')

plt.plot([30,110],[30,110],'k--')

plt.xlim(30,110)
plt.ylim(30,110)


df_students.plot.scatter(x='quiz (with drops)',y='course grade (with drops)')

plt.plot([30,110],[30,110],'k--')

plt.xlim(30,110)
plt.ylim(30,110)

In [ ]:
#mask = df['Student'] == 'Ramsdill, Payton'
#df[mask].columns
#
#for c in df[mask].columns:
#    print(df[mask][c])

In [ ]:
df_students
plt.figure(figsize=(16,6))

sns.violinplot(data=df_students)

plt.xticks(rotation=90);

In [ ]:
df_students
plt.figure(figsize=(16,6))

sns.boxplot(data=df_students[['course grade (with drops)','quiz (with drops)', 'hw (with drops)', 'final1 (with drops)', 'final2 (with drops)', 'lab (with drops)', 'reflection (with drops)']])

plt.xticks(rotation=45,ha='right', fontsize=18);

In [ ]:
plt.figure(figsize=(16,6))

bins = [0, 60,63,67,70,73,77,80,83,87,90,93,100 ]
grades = ['F','D-', 'D', 'D+', 'C-', 'C', 'C+', 'B-', 'B', 'B+','A-', 'A' ]

for b,grade in zip(bins, grades):
    mask = df_students['course grade (with drops)']>b
    sns.histplot(data=df_students[mask], x='course grade (with drops)',  bins=bins, label=grade)

plt.xticks(fontsize=18);
plt.xlabel('Class grades', fontsize=18)
plt.ylabel('Count', fontsize=18)

plt.legend()

#######################################
plt.figure(figsize=(16,6))
sns.histplot(data=df_students, x='course grade (with drops)', binrange=(40,100), bins=12)

plt.xticks(fontsize=18);
plt.xlabel('Class grades', fontsize=18)
plt.ylabel('Count', fontsize=18)

In [ ]:
sorting = 'course grade (with drops)'
#sorting = 'final2 (with drops)'

# PHYS 220, F23
df_students[['first name', 'last name','course grade (with drops)', \
             'final1 (with drops)', 'final2 (with drops)', \
             'quiz (with drops)', 'hw (with drops)']].sort_values(by=sorting)

# PHYS 260, F24
df_students[['first name', 'last name','course grade (with drops)', \
             'final1 (with drops)', 'final2 (with drops)', \
             'quiz (with drops)', 'hw (with drops)']].sort_values(by=sorting)

In [ ]:
sorting = 'last name'
df_students[['first name', 'last name','course grade (with drops)', \
             'final1 (with drops)', 'final2 (with drops)', \
             'quiz (with drops)', 'hw (with drops)']].sort_values(by=sorting)

In [ ]:
df_students

# Google Classroom

In [ ]:
infile = 'PHYS_260_Thermal_Physics_S23.csv'


df = pd.read_csv(infile)

In [ ]:
############# CLASSROOM #############
#print(df)

# Drop Enzo
df = df[~(df['Last Name']=='Morina')]
df = df[~(df['Last Name']=='Herron')]

df

In [ ]:
lnames = df['Last Name'].values
fnames = df['First Name'].values

print(lnames)
print(fnames)

In [ ]:
mask = (df['Last Name']=='Cestero') & (df['First Name']=='Anthony')

dft = df[mask]

print(dft)

In [ ]:
for fname,lname in zip(fnames,lnames):
    print(f'{fname} {lname}')
    print(f'{type(fname)} {type(lname)}')

    if type(lname)!=str or type(fname)!=str:
        print("Continuing")
        continue
    
    print(lname,fname)
    mask = (df['Last Name']==lname) & (df['First Name']==fname)
    print(mask)
    dftemp = df[mask]
    print(dftemp)
    print(df[mask])
    print()


In [ ]:
def get_assignments(df):
    
    maps = {'quiz':['QUIZ'], 'hw':['HW', 'HOMEWORK'], "participation":['PARTICIPATION', 'participation']}
    maps['final'] = ['FINAL EXAM']

    colnames = df.columns

    assignments = {"name":[], "points":[], "date":[], "type":[]}
    
    #assignments['part'] = {'name':[], 'date':[], 'points': []}
    #assignments['quiz'] = {'name':[], 'date':[], 'points': []}
    #assignments['hw'] = {'name':[], 'date':[], 'points': []}
    #assignments['midterm'] = {'name':[], 'date':[], 'points': []}
    #assignments['final'] = {'name':[], 'date':[], 'points': []}
    #assignments['extracredit'] = {'name':[], 'date':[], 'points': []}

    for name in colnames:
        #print(name, " -----")
        for key in maps.keys():
            #print(key)
            MATCH = False
            for val in maps[key]:
                #print(val)
                if name.find(val)>=0:
                    MATCH = True
            #print(MATCH,name,key,val)            
            if MATCH:
                points = df[name].iloc[1]
                #print(type(points))
                if type(points)!=str:
                    continue
                assignments['name'].append(name)
                assignments['type'].append(key)
                assignments['date'].append(df[name].iloc[0])
                assignments['points'].append(float(points))
    
    dfa = pd.DataFrame.from_dict(assignments)
    return dfa
            
'''
            hwandquizinfo[name] = []
        # EDAV
        elif name.find('HW')>=0 or name.find('HOMEWORK')>=0 or name.find('CLASS')>=0:
        # Quantum
        #elif name.find('HW')>=0 or name.find('HOMEWORK')>=0 or name.find('CODING')>=0:
            hwinfo[name] = []
            hwandquizinfo[name] = []
        elif name.find('PARTICIPATION')>=0 or name.find('PARTICIPATION')>=0:
            partinfo[name] = []
        elif name.find('EXTRA')>=0:
            extracreditinfo[name] = []
        elif name.find('MID')>=0:
            midterminfo[name] = []
        elif name.find('FINAL')>=0:
            finalinfo[name] = []
'''

dfa = get_assignments(df)

dfa


In [ ]:
mask = dfa['type']=='quiz'
dfa[mask]

In [ ]:
def represents_float(s):
    try: 
        float(s)
    except ValueError:
        return False
    else:
        return True

In [ ]:
dfa

In [ ]:
names = dfa['name']

dfplot = {"name":[], "scores":[], "type":[]}

for name in names:
    #print(name)
    dftemp = df[name]
    x = dftemp.values[2:] # Skip first two rows which are date and total points
    #print(x[0],represents_float(x[0]))
    if represents_float(x[0]):
        x = x.astype(float)
        #print(x)
        #print(len(x))
        if x[0]==x[0]:

            points = dfa[dfa['name']==name]['points'].values[0]
    
            
            dfplot["name"] += len(x)*[name]
            dfplot['scores'] += (x/points).tolist()
            
            atype = dfa[dfa['name']==name]['type'].values[0]
            dfplot['type'] += len(x)*[atype]

            #plt.figure()
            #plt.hist(x)

dfplot = pd.DataFrame.from_dict(dfplot)

In [ ]:
dfplot

In [ ]:
plt.figure(figsize=(16,4))

sns.violinplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

plt.xticks(rotation=90);

In [ ]:
plt.figure(figsize=(16,4))

sns.boxplot(data=dfplot, y="scores", x="name", hue='type');

plt.xticks(rotation=90);

In [ ]:
plt.figure(figsize=(16,4))

sns.boxplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

plt.plot([0],[0.8],'ro',markersize=10);

plt.xticks(rotation=270,ha='left');

In [ ]:
anames = np.unique(dfplot['name'])

plt.figure(figsize=(16,4))

sns.boxplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

xlabels = plt.gca().get_xticklabels(which='both')

n = 2

xpts = []
ypts = []

output = ""
for idx,xlabel in enumerate(xlabels):
    print(xlabel)
    name = xlabel.get_text()
    print(name)
    org_score = float(df.iloc[n][name])
    points = dfa[dfa['name']==name]['points'].values[0]
    score = org_score/points
    xpts.append(name)
    ypts.append(score)
    
    
lname = df.iloc[n]['Last Name']
fname = df.iloc[n]['First Name']
email = df.iloc[n]['Email Address']

print()

plt.plot(xpts,ypts,'ro-',markersize=10, label=f'{fname} {lname}');

plt.xticks(rotation=270,ha='left');
plt.legend()





In [ ]:
#xlabel.

In [ ]:
df.iloc[2]['Last Name']

In [ ]:
df.iloc[2]

https://medium.com/dunder-data/automatically-wrap-graph-labels-in-matplotlib-and-seaborn-a48740bc9ce

In [ ]:
import textwrap
def wrap_labels(ax, width, break_long_words=False):
    labels = []
    for label in ax.get_xticklabels():
        text = label.get_text()
        labels.append(textwrap.fill(text, width=width,
                      break_long_words=break_long_words))
    ax.set_xticklabels(labels, rotation=270)

In [ ]:
weighting = {"quiz":0.45, "hw": 0.20, "participation":0.05, "final": 0.30}
weighting_sum = 0
for key in weighting.keys():
    weighting_sum += weighting[key]
    
print(f"Weighting sum: {weighting_sum}")


In [ ]:
dfa

In [ ]:
n = 2

dfstudent = df.iloc[2]

dfstudent

In [ ]:
n = 12

dfstudent = df.iloc[n]

#print(dfstudent)
#print()

nassignments = len(dfa)

scores = {}
for key in weighting.keys():
    scores[key] = []
print(scores)

for n in range(nassignments):
    aname = dfa.iloc[n]['name']
    atype = dfa.iloc[n]['type']
    points = dfa.iloc[n]['points']
    date = dfa.iloc[n]['date']
    
    org_score = float(dfstudent[aname])
    if org_score != org_score:
        continue
        
    score = org_score / points
    scores[atype].append(score)
    
    
print(scores)
dfstudent

In [ ]:
def average_scores(scores, weighting, drop_lowest=None):
    weight_sum = 0
    what_was_included = []
    
    averages = {}
    
    if drop_lowest is None:
        drop_lowest = {}
        for key in weighting.keys():
            drop_lowest[key] = 0
    
    tot = 0
    
    #if drop_lowest:
    #    scores = n
    
    for key in scores:
        if len(scores[key])>0:
            if key not in what_was_included:
                weight_sum += weighting[key]
                what_was_included.append(key)
            scores_to_use = scores[key]
            #print(drop_lowest)
            #print(key)
            #print(scores_to_use)
            if drop_lowest[key]>0:
                scores_to_use = np.sort(scores_to_use)[drop_lowest[key]:]
            ave = np.average(scores_to_use)
            w = weighting[key]
            
            averages[key] = ave
            
            #print(key,scores_to_use, ave, w)

            tot += ave*w
    
    #print(tot, weight_sum)
    tot /= weight_sum
    #print(tot)
    return tot, averages

In [ ]:
#x = [1, 10, 3, 4]
#np.sort(x)[1:]

In [ ]:
drops = {"quiz":1, "hw": 1, "participation":0, "final": 0}


print(average_scores(scores,weighting))

print("---------------")

print(average_scores(scores,weighting, drop_lowest=drops))

# Reference
https://julien.danjou.info/sending-emails-in-python-tutorial-code-examples/

https://realpython.com/python-send-email/


In [ ]:
################################################################################
def write_and_send_email(sender_email, rec_email, title, name, body_text, subject, attachment=None):

    message = MIMEMultipart("alternative")
    message["Subject"] = subject
    message["From"] = sender_email
    message["To"] = rec_email

    message["Cc"] = "mbellis@siena.edu"

    text = ""
    text += body_text

    part1 = MIMEText(text, "plain")

    message.attach(part1)
    
    # Attachment
    # https://stackoverflow.com/questions/69901601/send-email-with-attachment-using-python-3-7-3
    if attachment is not None:
        filename = attachment
        with open(filename, "rb") as fs:
            part = MIMEBase('application', "octet-stream")
            part.set_payload(fs.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment', filename=os.path.basename(filename))
        message.attach(part)


    context = ssl.create_default_context()
    
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, password)

        # UNCOMMENT THIS WHEN SENDING FOR REAL
        receiver_email = rec_email

        server.sendmail(sender_email, receiver_email, message.as_string())





In [ ]:
df

In [ ]:
# Make a directory if we need to for the figures
try:
   os.makedirs("figures")
except FileExistsError:
   # directory already exists
   pass


####################################
# For mailing

########################################################################
login = "1a2b3c4d5e6f7g" # paste your login generated by Mailtrap
password = "1a2b3c4d5e6f7g" # paste your password generated by Mailtrap
sender_email = "matthew.bellis@gmail.com"
port = 465
password = getpass.getpass("Your password: ")

###############################################

max_names = 17
#max_names = 3

drops = {"quiz":1, "hw": 1, "participation":0, "final": 0}

course_grades = []

for n in range(2,max_names):
#for n in range(16,17):

    plt.figure(figsize=(16,6), dpi=200, facecolor='white')

    sns.boxplot(data=dfplot, y="scores", x="name", hue='type', dodge=False);

    xlabels = plt.gca().get_xticklabels(which='both')

    xpts = []
    ypts = []
    
    dfstudent = df.iloc[n]

    scores = {}
    for key in weighting.keys():
        scores[key] = []
    #print(scores)
    
    running_aves = []
    all_averages = []
    
    lname = dfstudent['Last Name']
    fname = dfstudent['First Name']
    email = dfstudent['Email Address']

    subject = f"PHYS 260 - Thermal Physics: summary grades for {fname} {lname}"
    email_body = f"Summary grades for {fname} {lname} ({email})\n\n"
    
    #email_body += '<html><font="Fixed Width">\n'
    
    for idx,xlabel in enumerate(xlabels):
        #print(xlabel)
        name = xlabel.get_text()
        #print(name)
        org_score = float(dfstudent[name])
        points = dfa[dfa['name']==name]['points'].values[0]
        score = org_score/points
        xpts.append(name)
        ypts.append(score)
        
        #print(xlabel)
        atype = dfa[dfa['name']==xlabel.get_text()]['type'].values[0]
        date = dfa[dfa['name']==xlabel.get_text()]['date'].values[0]

        #print(atype,date)
        scores[atype].append(score)
        running_ave, all_averages_temp = average_scores(scores,weighting)#, drop_lowest=drops)
        running_aves.append(running_ave)
        all_averages.append(all_averages_temp)

        
        #email_body += f"{name:30s} {atype:10s} {date:15s} {100*score:5.2f} ({org_score}/{points})\n"   
        email_body += f"{100*score:6.2f} ({org_score:5.1f}/{points:5.1f})  {atype:10s} {date:15s} {name:30s} \n"   

        
    #print(f"{fname} {lname} {email}")
    #print(output)
    #email_body += f"{aname} {atype} {points} {date} {score}\n")
    
    #course_grade = running_aves[-1]
    #print(all_averages[-1])
    
    #email_body += '</font></html>'

    course_grade, final_all_averages = average_scores(scores,weighting, drop_lowest=drops)
    #print(final_all_averages)
    email_body += "\n"
    email_body += f"Participation grade:              {100*final_all_averages['participation']:6.2f}%\n"
    email_body += f"Homework grade: (lowest dropped)  {100*final_all_averages['hw']:6.2f}%\n"
    email_body += f"Quiz grade:     (lowest dropped)  {100*final_all_averages['quiz']:6.2f}%\n"
    email_body += f"Final exam grade:                 {100*final_all_averages['final']:6.2f}%\n"
    
    
    course_grades.append(course_grade)
    
    email_body += f"\nCourse grade: {100*course_grade:8.2f}%\n"


    print(email_body)

    plt.plot(xpts,ypts,'k*',markersize=30, linewidth=5);    
    plt.plot(xpts,ypts,'r*',markersize=20, linewidth=5, label=f'{lname}: Assignment grades');
    plt.plot(xpts,running_aves,'ko--',markersize=10, linewidth=5, label=f'{lname}: Course grade {100*course_grade:0.1f}');

    #plt.xticks(rotation=270,ha='center',fontsize=14);
    #plt.xticks(rotation=90,ha='center',fontsize=14);
    plt.legend()
    wrap_labels(plt.gca(), 23)
    plt.xlabel('')
    plt.ylabel('Assignment grade',fontsize=14)
    plt.gca().set_facecolor("white")

    plt.tight_layout()
    
    outfilename = f"figures/PHYS_260_summary_{fname}_{lname}.png"
    plt.savefig(outfilename)

    rec_email = email
    # For testing
    #rec_email = 'mbellis@siena.edu'
    #write_and_send_email(sender_email=sender_email, rec_email=rec_email, title=None, name=None, \
    #                     subject=subject, body_text=email_body, attachment=outfilename)

In [ ]:
np.sort(df['FINAL EXAM'].values[1:].astype(float)/260)

In [ ]:
scores = {}
for key in weighting.keys():
    scores[key] = []
print(scores)

nassignments = len(dfa)

for n in range(nassignments):
    aname = dfa.iloc[n]['name']
    atype = dfa.iloc[n]['type']
    points = dfa.iloc[n]['points']
    date = dfa.iloc[n]['date']
    
    
    org_score = float(dfstudent[aname])
    if org_score != org_score:
        continue
        
    score = org_score / points
    print(f"{aname} {atype} {points} {date} {score}")
    
    scores[atype].append(score)
    x,y = average_scores(scores,weighting)
    print(f"Running average: {100*x:.2f}")
    
print(scores)
    

In [ ]:
x = np.sort(100*np.array(course_grades))

for i in x:
    print(i)